In [1]:
import z3
import SATfunctions2 as SATf
from typing import List
from math import log,ceil

def read_input_file(file_path):
    with open(file_path, 'r') as file:
        m = int(file.readline().strip())
        n = int(file.readline().strip())
        #s = [0 for _ in range(m)]
        s = list(map(int, file.readline().strip().split()))
        w = list(map(int, file.readline().strip().split()))

        # Inizializza la matrice delle distanze D
        D = [[0 for _ in range(n+1)] for _ in range(n+1)]

        # Leggi i valori della matrice delle distanze dalla restante parte del file
        for i in range(n+1):
            row_values = list(map(int, file.readline().strip().split()))
            D[i] = row_values

    #U vettore di n variabili (1 e n) con valore da 1 a n

    return m, n, s, w, D

In [2]:
def SAT_mcp(m, n, s, w, D):
    mb = SATf.int2boolList(m)
    nb = SATf.int2boolList(n)
    sb = [SATf.int2boolList(s[i]) for i in range(m)]
    wb = [SATf.int2boolList(w[i]) for i in range(n)]
    Db = [[SATf.int2boolList(D[i][j]) for j in range(n+1)] for i in range(n+1)]
    # Inizializza il solver Z3
    solver = z3.Solver()

    x = [[z3.Bool(f'x_{i}_{j}') for j in range(len(mb))] for i in range(n)]
    tour = [[z3.Bool(f'tour_{i}_{j}') for j in range(len(nb))] for i in range(n)]
    #Domains
    for i in range(n):
        solver.add(SATf.at_least_one_T(x[i]))
        solver.add(SATf.at_least_one_T(tour[i]))
        solver.add(SATf.lte(x[i],mb))
        solver.add(SATf.lte(tour[i],SATf.sum_b_list([SATf.eq(x[i],x[k]) for k in range(n)])))

    #C1     weight constraint
    for i in range(m):
        solver.add(SATf.gte(sb[i],SATf.sum_b_list([SATf.enable(wb[j],SATf.eq(SATf.int2boolList(i+1),x[j])) for j in range(n)])))
        solver.add(SATf.gte(sb[0],SATf.sum_b_list([SATf.enable(wb[0],SATf.eq(SATf.int2boolList(i+1),x[0]))])))
    
    #C2   a courier cannot delivery two items at the same time  
    for i in range(n):
        for j in range(i+1,n):
            solver.add(z3.Or(SATf.ne(x[i],x[j]),SATf.ne(tour[i],tour[j])))

    #loss
    lb = min(D[n])+min([D[i][n] for i in range(n+1)])
    ub = sum([max(D[i]) for i in range(n+1)])
    dist = [[z3.Bool(f'loss_{i}_{j}') for j in range(ceil(log(ub,2)))] for i in range(m)]    #list of distances of each courier





    if solver.check() == z3.sat:
        model = solver.model()
        print("Sat:")
        print(model)
    else:
        print("Unsat")

In [5]:
m,n,s,w,D = read_input_file('instance.txt')
print(m,n,s,w,D)
SAT_mcp(m, n, s, w, D)

1 4 [10] [1, 1, 1, 1] [[0, 4, 4, 3, 2], [4, 0, 4, 4, 4], [4, 4, 0, 3, 2], [3, 4, 3, 0, 3], [2, 4, 2, 3, 0]]
Sat:
[x_3_0 = True,
 tour_1_1 = False,
 xge_6b019cd1-73c7-476c-8f9e-dede00000806 = False,
 tour_2_0 = False,
 xge_6085250a-3bd5-4889-b884-806332d0ea50 = True,
 xge_a2f53f2f-28d0-4d14-bdfc-811dcaae7985 = False,
 xge_f477a1da-3238-426d-8f52-5e8cb5db949b = False,
 tour_0_0 = False,
 tour_1_0 = True,
 tour_3_0 = False,
 tour_3_1 = False,
 xge_be7d8409-0c87-4ead-8bac-ac938ce848b6 = False,
 xge_1c01e7e7-f4f2-4e6f-a554-a8a88415dbd6 = True,
 xge_a7a43381-e5b1-4c95-9312-c566f41affaa = False,
 tour_2_2 = False,
 x_0_0 = True,
 x_1_0 = True,
 tour_0_1 = True,
 x_2_0 = True,
 tour_1_2 = False,
 xge_acc53615-dc33-4dc0-b44b-01fab168456b = False,
 xge_95f4fdd0-7e18-44ba-9e81-ff657255a375 = False,
 tour_3_2 = True,
 xge_c00221e0-791a-42ff-adb0-3b958e2963c9 = False,
 tour_0_2 = True,
 tour_2_1 = True,
 xge_8759c385-2ea1-4c9f-a493-74deb072e6ea = False]
